<!-- #### Import Julia packages -->

In [1]:
# import Pkg
# Pkg.add("JSON3")
# Pkg.add("Interpolations")

In [55]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
using Makie
# using Interpolations

<!-- #### Import Custom functions -->

In [56]:
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")
include("Julia_functions/post_processing.jl")

get_control_int (generic function with 1 method)

<!-- #### Read JSON File -->

In [57]:
width, height, Tini_json, Tfin_json, windows, doors, heating_elem = get_room("Boundary.json")

(5, 3, 293.15, 293.15, Any[(0.415, 1.965, "Top"), (3.095, 4.635, "Bottom")], Any[(2.15125, 2.65125, "Left")], [[0.7139999389648437, 1.4039999389648437, 0.17925003051757812, 0.23925003051757812], [4.7339999389648435, 4.783999938964843, 1.9792500305175782, 2.5392500305175782]])

In [58]:
# price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")
price_EUR, price_NOK, T_out, hour_lst = get_price_temp("WeatherandEnergy.json")

(Any[0.00809, 0.00803, 0.01533, 0.00809, 0.00801, 0.00802, 0.012, 0.01261, 0.01468], Any[0.09767, 0.09694, 0.18507, 0.09767, 0.0967, 0.09682, 0.14487, 0.15223, 0.17722], Any[277.04999999999995, 277.25, 277.15, 276.75, 276.15, 274.54999999999995, 271.84999999999997, 270.25, 269.95], Any[13, 14, 15, 16, 17, 18, 19, 20, 21])

<!-- #### Boundary conditions -->

In [59]:
f_Γ_w = generate_f_Γ(windows, width, height)
f_Γ_d = generate_f_Γ(doors, width, height)
f_Γ_wall(x) = !(f_Γ_w(x) || f_Γ_d(x))

f_Γ_wall (generic function with 1 method)

<!-- #### Domain & Discretization -->

In [60]:
domain = (0, width , 0, height)
partition = (40,40)
model = CartesianDiscreteModel(domain,partition)
update_labels!(1, model, f_Γ_w, "window")
update_labels!(2, model, f_Γ_d, "door")
update_labels!(3, model, f_Γ_wall, "wall")

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_w = BoundaryTriangulation(model, tags="window")
dΓ_w = Measure(Γ_w, degree)

Γ_d = BoundaryTriangulation(model, tags="door")
dΓ_d = Measure(Γ_d, degree)

Γ_wall = BoundaryTriangulation(model, tags="wall")
dΓ_wall = Measure(Γ_wall, degree)

reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)

# fig = CairoMakie.plot(Ω)
# CairoMakie.wireframe!(Ω, color=:black, linewidth=1)                        # add triangulation
# CairoMakie.save("grid.png", fig)

# display(fig)


UnconstrainedFESpace()

<!-- #### Time parameters -->

In [61]:
t0 = 0.0    # Start time
tF = 28800   # End time
Δt = 60

60

<!-- #### Room parameters -->

In [62]:
h_wall(x) = 0.22*0.01           # [W/m^2K]
h_window(x) = 1.2*0.01
h_door(x) = 1.0*0.01 #mindre?

ρ(x)=1.225
c(x)=1020.0
k(x)=5#3.5


h = (h_wall, h_window, h_door) 
constants = (c, ρ, k, h) 


Tout = get_temp_function(T_out, tF)

Tini(x)= Tini_json
TFin(x)= Tfin_json

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(TFin, Uspace(tF))

SingleFieldFEFunction():
 num_cells: 1600
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 8491082492046673767

<!-- #### Controll parameter -->

In [ ]:
q_pos = heating_elements(heating_elem)

Q(x,t)=0*q_pos(x)
Qt(t)=x->Q(x,t)

Qt (generic function with 1 method)

<!-- #### Solver parameters -->

In [ ]:
L2norm(u)=√(Δt*((∑(∫(u[1][2]⋅u[1][2])*dΩ)+∑(∫(u[end][2]⋅u[end][2])*dΩ))/2 + ∑(∑(∫(u[k][2]⋅u[k][2])*dΩ) for k=2:length(u)-1)))
L2skp(u)=Δt*((∑(∫(u[1][2]⋅u[1][2])*dΩ)+∑(∫(u[end][2]⋅u[end][2])*dΩ))/2 + ∑(∑(∫(u[k][2]⋅u[k][2])*dΩ) for k=2:length(u)-1))
get_control_int(qs) = [∑(∫(u)*dΩ) for (t,u) in qs] ./ ∑(∫(q_pos)*dΩ)
get_temperature_int(Ts) = [∑(∫(temp)*dΩ) for (t,temp) in Ts] ./ ∑(∫(1)*dΩ)

println(∑(∫(q_pos)*dΩ))
Proj(a,b,z) = min(max(a,z),b)
a=0.0
b=500
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]

0.06328124999999997
500


Proj (generic function with 2 methods)

<!-- #### Test Gradient DEs -->

In [50]:
# get_control_int(qs) = [∑(∫(u)*dΩ) for (t,u) in qs] ./ ∑(∫(q_pos)*dΩ)
# get_temperature_int(Ts) = [∑(∫(temp)*dΩ) for (t,temp) in Ts] ./ ∑(∫(1)*dΩ)


# function real_price(qs,price)
#     price1 = 0
#     for (t,QQ) in qs
#         price1 += Δt*price(t)*∑(∫(QQ)*dΩ)/3600/1000
#     end
#     return price1
# end

# price = get_price_function(price_NOK, tF, 1)
# println(real_price(qs,price))
# # println(get_temperature_int(Ts-Tfin))


function result_plot(;T_int, Q_int, t0, tF, hour_lst, price_NOK, T_out)
    time = LinRange(t0, tF, length(T_int))

    fig = CairoMakie.Figure(size=(600,900))

    ax = CairoMakie.Axis(fig[1,1], title="Temperature profile", xlabel="Time (hours)", ylabel="Temperature (°C)")
    lines!(ax, time, T_int, color=:green) #label="Temp"

    ax=CairoMakie.Axis(fig[2,1], title="Optimal Heating Schedule", xlabel="Time (hours)", ylabel="Heating Output (W)")  
    lines!(ax, time, Q_int, color=:red)

    ax=CairoMakie.Axis(fig[3,1], title="Energy price over time", xlabel="Time (hours)", ylabel="Price (NOK)")
    lines!(ax, hour_lst, price_NOK, color=:blue)

    ax=CairoMakie.Axis(fig[4,1], title="Outside Temperature over time", xlabel="Time (hours)", ylabel="Temperature (°C)")
    lines!(ax, hour_lst, T_out, color=:orange)

    return fig
end

function result_plot2(; T_int, Q_int, t0, tF, hour_lst, price_NOK, T_out)
    # time = LinRange(t0, tF, length(T_int))
    time = LinRange(t0, tF, length(T_int)) ./ 3600  # convert from seconds to hours
    time2 = LinRange(t0, tF, length(T_int))

    
    time_price = [price(t) for t in time2]

    fig = CairoMakie.Figure(size=(600, 900), figure_padding=(0, 5, 0, 5))

    # Formatter for temperature axis (Kelvin to Celsius)
    temp_formatter = v -> ["$(round(vi - 273.15, digits=1))" for vi in v]

    # Temperature profile
    ax = CairoMakie.Axis(
        fig[1, 1],
        title = "Temperature profile",
        xlabel = "Time (hours)",
        ylabel = "Temperature (°C)",
        ytickformat = temp_formatter
    )
    lines!(ax, time, T_int, color = :green)

    # Heating output
    ax = CairoMakie.Axis(
        fig[2, 1],
        title = "Optimal Heating Schedule",
        xlabel = "Time (hours)",
        ylabel = "Total heating Output (W)"
    )
    lines!(ax, time, Q_int, color = :red)

    # Energy price
    ax = CairoMakie.Axis(
        fig[3, 1],
        title = "Energy price over time",
        xlabel = "Time (hours)",
        ylabel = "Price (NOK)"
    )
    lines!(ax, time, time_price, color = :blue)

    # Outside temperature
    ax = CairoMakie.Axis(
        fig[4, 1],
        title = "Outside Temperature over time",
        xlabel = "Time (hours)",
        ylabel = "Temperature (°C)",
        ytickformat = temp_formatter
    )
    lines!(ax, hour_lst .-10, T_out, color = :orange)

    return fig
end

function plot_costs(costs)
    fig = CairoMakie.Figure(size=(600, 300))

    ax = CairoMakie.Axis(
        fig[1, 1],
        xlabel = "Iteration",
        ylabel = "Cost"#,
        # yscale = log10  # Make y-axis logarithmic
    )

    lines!(ax, 1:length(costs), costs, color = :purple)

    return fig
end

function plot_costs2(costs)
    fig = CairoMakie.Figure(size=(600, 300))

    ax = CairoMakie.Axis(
        fig[1, 1],
        xlabel = "Iteration",
        ylabel = "Cost",
        yscale = log10  # Make y-axis logarithmic
    )

    lines!(ax, 1:length(costs), costs, color = :purple)

    return fig
end


plot_costs2 (generic function with 1 method)

In [51]:
price = t -> 0

γ = 1.225*1020
Q(x,t)= 0
Qt(t)=x->Q(x,t)

(Ts,qs,Ws,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=0,
armijoparas=(ρ=0.5, α_0=25, α_min=1e-3, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed


(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (120.0, SingleFieldFEFunction()), (180.0, SingleFieldFEFunction()), (240.0, SingleFieldFEFunction()), (300.0, SingleFieldFEFunction()), (360.0, SingleFieldFEFunction()), (420.0, SingleFieldFEFunction()), (480.0, SingleFieldFEFunction()), (540.0, SingleFieldFEFunction())  …  (28260.0, SingleFieldFEFunction()), (28320.0, SingleFieldFEFunction()), (28380.0, SingleFieldFEFunction()), (28440.0, SingleFieldFEFunction()), (28500.0, SingleFieldFEFunction()), (28560.0, SingleFieldFEFunction()), (28620.0, SingleFieldFEFunction()), (28680.0, SingleFieldFEFunction()), (28740.0, SingleFieldFEFunction()), (28800.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (120.0, SingleFieldFEFunction()), (180.0, SingleFieldFEFunction()), (240.0, SingleField

In [52]:
price = t -> 0
c1_zero = (plot_costs2(costs))
CairoMakie.save("price_zero_convergence.png", c1_zero)


Q_int = get_control_int(qs)
T_int = get_temperature_int(Ts)
c2_zero = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_zero_result.png", c2_zero)

CairoMakie.Screen{IMAGE}


In [ ]:
price = t -> 0.4 + χ(t,-1,3000)*2 + χ(t,10000,15000)*2 + χ(t,23000,28800+1)*2
γ = 1.225*1020*2

(Ts2,qs2,Ws2,costs2) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=300,
armijoparas=(ρ=0.5, α_0=10, α_min=1e-4, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

In [ ]:
price = t -> 0.4 + χ(t,-1,3000)*2 + χ(t,10000,15000)*2 + χ(t,23000,28800+1)*2

# c1_step = (plot_costs(costs2))
c1_step = (plot_costs2(costs2))

CairoMakie.save("price_step_convergence.png", c1_step)


Q_int = get_control_int(qs2)
T_int = get_temperature_int(Ts2)
c2_step = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_step_result.png", c2_step)



In [27]:
price = get_price_function(price_NOK, tF, 1)
γ = 1.225*1020*3/4

(Ts3,qs3,Ws3,costs3) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=30,
armijoparas=(ρ=0.5, α_0=30, α_min=1e-3, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=1.1375494414607415e6, k = 1
α = 30, new_cost = 540314.0760077413, L2fgrad = 3872.304922810362, interm = 0.0
entered for loop, E=540314.0760077413, k = 2
α = 30, new_cost = 270463.00994170835, L2fgrad = 2601.2988519779137, interm = 0.0
entered for loop, E=270463.00994170835, k = 3
α = 30, new_cost = 146758.83542353797, L2fgrad = 1761.5104347260124, interm = 0.0
entered for loop, E=146758.83542353797, k = 4
α = 30, new_cost = 91012.96816254285, L2fgrad = 1180.0877387428802, interm = 0.0
entered for loop, E=91012.96816254285, k = 5
α = 30, new_cost = 65214.63641761725, L2fgrad = 802.4749828185815, interm = 0.0
entered for loop, E=65214.63641761725, k = 6
α = 30, new_cost = 53583.55311515291, L2fgrad = 535.0268739275165, interm = 0.0
entered for loop, E=53583.55311515291, k = 7
α = 30, new_cost = 48060.125972329966, L2fgrad = 366.9890710952898, interm = 0.0
entered for loop, E=48060.125972329966, k = 8
α = 30, new_cost = 45512.209349214376, L2fgrad

(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (120.0, SingleFieldFEFunction()), (180.0, SingleFieldFEFunction()), (240.0, SingleFieldFEFunction()), (300.0, SingleFieldFEFunction()), (360.0, SingleFieldFEFunction()), (420.0, SingleFieldFEFunction()), (480.0, SingleFieldFEFunction()), (540.0, SingleFieldFEFunction())  …  (28260.0, SingleFieldFEFunction()), (28320.0, SingleFieldFEFunction()), (28380.0, SingleFieldFEFunction()), (28440.0, SingleFieldFEFunction()), (28500.0, SingleFieldFEFunction()), (28560.0, SingleFieldFEFunction()), (28620.0, SingleFieldFEFunction()), (28680.0, SingleFieldFEFunction()), (28740.0, SingleFieldFEFunction()), (28800.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (120.0, SingleFieldFEFunction()), (180.0, SingleFieldFEFunction()), (240.0, SingleField

In [24]:
price = get_price_function(price_NOK, tF, 1)

# c1_real = (plot_costs(costs3))
c1_real = (plot_costs2(costs3))

CairoMakie.save("price_real_convergence.png", c1_real)

Q_int = get_control_int(qs3)
T_int = get_temperature_int(Ts3)
c2_real = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_real_result.png", c2_real)


CairoMakie.Screen{IMAGE}


In [76]:
price = t -> 1
γ = 1.225*1020*2

(Ts4,qs4,Ws4,costs4) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
Q=qs4, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=false, 
tol=1e-5, 
iter_max=30,
armijoparas=(ρ=0.5, α_0=15, α_min=1e-3, σ=0.0), 
Δt=Δt, 
t0=t0,
tF=tF,
Tout=Tout,
constants=constants,
Tfin=Tfin,
q_pos=q_pos)

T computed
W computed
entered for loop, E=263833.8065285998, k = 1
α = 15, new_cost = 263772.2814226481, L2fgrad = 246.14883093214746, interm = 0.0
entered for loop, E=263772.2814226481, k = 2
α = 15, new_cost = 263712.40886606, L2fgrad = 246.1143181169023, interm = 0.0
entered for loop, E=263712.40886606, k = 3
α = 15, new_cost = 263653.15423493757, L2fgrad = 246.29870334370415, interm = 0.0
entered for loop, E=263653.15423493757, k = 4
α = 15, new_cost = 263594.9141208321, L2fgrad = 246.26333789051654, interm = 0.0
entered for loop, E=263594.9141208321, k = 5
α = 15, new_cost = 263537.73355236324, L2fgrad = 246.33731952062837, interm = 0.0
entered for loop, E=263537.73355236324, k = 6
α = 15, new_cost = 263480.7061604707, L2fgrad = 246.42706573949866, interm = 0.0
entered for loop, E=263480.7061604707, k = 7
α = 15, new_cost = 263425.7496422412, L2fgrad = 246.42015647749741, interm = 0.0
entered for loop, E=263425.7496422412, k = 8
α = 15, new_cost = 263371.027657206, L2fgrad = 246.6

(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (120.0, SingleFieldFEFunction()), (180.0, SingleFieldFEFunction()), (240.0, SingleFieldFEFunction()), (300.0, SingleFieldFEFunction()), (360.0, SingleFieldFEFunction()), (420.0, SingleFieldFEFunction()), (480.0, SingleFieldFEFunction()), (540.0, SingleFieldFEFunction())  …  (28260.0, SingleFieldFEFunction()), (28320.0, SingleFieldFEFunction()), (28380.0, SingleFieldFEFunction()), (28440.0, SingleFieldFEFunction()), (28500.0, SingleFieldFEFunction()), (28560.0, SingleFieldFEFunction()), (28620.0, SingleFieldFEFunction()), (28680.0, SingleFieldFEFunction()), (28740.0, SingleFieldFEFunction()), (28800.0, SingleFieldFEFunction())], Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (60.0, SingleFieldFEFunction()), (120.0, SingleFieldFEFunction()), (180.0, SingleFieldFEFunction()), (240.0, SingleField

In [77]:
price = t -> 1

# c1_const = (plot_costs(costs4))
c1_const = (plot_costs2(costs4))

CairoMakie.save("price_const_convergence.png", c1_const)

Q_int = get_control_int(qs4)
T_int = get_temperature_int(Ts4)
c2_const = result_plot2(;T_int,Q_int,t0,tF,hour_lst,price_NOK,T_out)
CairoMakie.save("price_const_result.png", c2_const)


CairoMakie.Screen{IMAGE}


In [72]:
# Opprett en mappe for midlertidige filer hvis den ikke eksisterer
if !isdir("tmp")
    mkdir("tmp")
end

# Initialtilstanden og resten av tidsløsningene
# uh0 = solution[1][2]  # Første element er (t0, T0), vi henter T0
# uh  = solution[2:end]  # De resterende løsningene


uh = Ts4
# Lagre resultater i Paraview-format (VTK)
createpvd("resultsOutisde") do pvd
    # Første løsning (t=0)
    # pvd[0] = createvtk(Ω, "tmp/results_0.vtu", cellfields=["u" => uh0])

    # Lagrer løsninger for hver tidssteg
    count = 0
    for (tn, uhn) in uh
        if count%6 == 0
            pvd[tn] = createvtk(Ω, "tmp/results_$count.vtu", cellfields=["u" => uhn])
        end
        count+=1
    end
end


82-element Vector{String}:
 "resultsOutisde.pvd"
 "tmp/results_0.vtu"
 "tmp/results_6.vtu"
 "tmp/results_12.vtu"
 "tmp/results_18.vtu"
 "tmp/results_24.vtu"
 "tmp/results_30.vtu"
 "tmp/results_36.vtu"
 "tmp/results_42.vtu"
 "tmp/results_48.vtu"
 ⋮
 "tmp/results_432.vtu"
 "tmp/results_438.vtu"
 "tmp/results_444.vtu"
 "tmp/results_450.vtu"
 "tmp/results_456.vtu"
 "tmp/results_462.vtu"
 "tmp/results_468.vtu"
 "tmp/results_474.vtu"
 "tmp/results_480.vtu"